In [1]:
import evaluate
import numpy as np
from datasets import load_from_disk, disable_caching
from sklearn.metrics import f1_score
from transformers import (
    AutoTokenizer,
    BertForSequenceClassification,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
    set_seed,
)

In [2]:
# import random
# seeds = [random.randint(0, 1e9) for _ in range(5)]
# seeds

In [3]:
disable_caching()

In [4]:
num_labels = 5
id2label = {
    0: "reject",
    1: "B_supplies_A",
    2: "A_supplies_B",
    3: "ambiguous",
    4: "ownership",
}
label2id = {
    "reject": 0,
    "B_supplies_A": 1,
    "A_supplies_B": 2,
    "ambiguous": 3,
    "ownership": 4,
}
metric = evaluate.load("f1")

In [5]:
model_name = "google-bert/bert-base-uncased" #uncased
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens(
    {"additional_special_tokens": ["__NE_FROM__", "__NE_TO__", "__NE_OTHER__"]}
)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [6]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1_micro = f1_score(labels, predictions, average='micro')
    f1_macro = f1_score(labels, predictions, average='macro')
    f1_classwise = f1_score(labels, predictions, average=None)

    return {
        "f1_micro": f1_micro,
        "f1_macro": f1_macro,
        **{f"f1_class_{i}": score for i, score in enumerate(f1_classwise)}
    }


def model_init():
    model = BertForSequenceClassification.from_pretrained(
        model_name,
        num_labels=5,
        id2label=id2label,
        label2id=label2id,
    )
    model.resize_token_embeddings(len(tokenizer))
    return model

In [8]:
ds = load_from_disk("../../datasets/ManualDataset")
ds = ds.select_columns(["masked_text", "label"])
ds = ds.rename_column("masked_text", "text")
ds = ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/2559 [00:00<?, ? examples/s]

Map:   0%|          | 0/418 [00:00<?, ? examples/s]

Map:   0%|          | 0/745 [00:00<?, ? examples/s]

In [9]:
def run_experiment(seed):
    set_seed(seed)
    training_args = TrainingArguments(
        seed=seed,
        data_seed=seed,
        tf32=True,
        output_dir="logs/experiment_1_bert",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=10,
        weight_decay=0.01,
        eval_strategy="epoch",
        save_strategy="epoch",
        warmup_ratio=0.1,
        load_best_model_at_end=True,
        save_total_limit=1,
        report_to=[],
        save_only_model=True,
    )
    trainer = Trainer(
        model_init=model_init,
        args=training_args,
        train_dataset=ds["train"],
        eval_dataset=ds["valid"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )
    trainer.train()
    test_results = trainer.predict(ds["test"])
    return test_results

In [10]:
seeds = [947706532, 219089540, 567388559, 220752674, 837810645]
all_results = []

for seed in seeds:
    results = run_experiment(seed)
    all_results.append(results)

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

  0%|          | 0/1600 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

{'eval_loss': 1.0666015148162842, 'eval_f1_micro': 0.6339712918660287, 'eval_f1_macro': 0.4096190608483982, 'eval_f1_class_0': 0.7333333333333333, 'eval_f1_class_1': 0.0, 'eval_f1_class_2': 0.6837060702875399, 'eval_f1_class_3': 0.45714285714285713, 'eval_f1_class_4': 0.17391304347826086, 'eval_runtime': 0.3688, 'eval_samples_per_second': 1133.484, 'eval_steps_per_second': 73.215, 'epoch': 1.0}


  0%|          | 0/27 [00:00<?, ?it/s]

...

  0%|          | 0/27 [00:00<?, ?it/s]

{'eval_loss': 0.8165282011032104, 'eval_f1_micro': 0.8277511961722488, 'eval_f1_macro': 0.7997075908542233, 'eval_f1_class_0': 0.8414634146341463, 'eval_f1_class_1': 0.7878787878787878, 'eval_f1_class_2': 0.8487084870848709, 'eval_f1_class_3': 0.8062015503875969, 'eval_f1_class_4': 0.7142857142857143, 'eval_runtime': 0.3027, 'eval_samples_per_second': 1381.04, 'eval_steps_per_second': 89.206, 'epoch': 9.0}
{'loss': 0.1095, 'grad_norm': 0.9014012813568115, 'learning_rate': 1.3888888888888892e-06, 'epoch': 9.38}


  0%|          | 0/27 [00:00<?, ?it/s]

{'eval_loss': 0.8534265756607056, 'eval_f1_micro': 0.8205741626794258, 'eval_f1_macro': 0.7937412014796928, 'eval_f1_class_0': 0.8373493975903614, 'eval_f1_class_1': 0.78125, 'eval_f1_class_2': 0.835820895522388, 'eval_f1_class_3': 0.8, 'eval_f1_class_4': 0.7142857142857143, 'eval_runtime': 0.3114, 'eval_samples_per_second': 1342.13, 'eval_steps_per_second': 86.693, 'epoch': 10.0}
{'train_runtime': 83.0713, 'train_samples_per_second': 308.049, 'train_steps_per_second': 19.261, 'train_loss': 0.44194120734930037, 'epoch': 10.0}


  0%|          | 0/47 [00:00<?, ?it/s]

In [11]:
# Calculate mean and std of F1 scores
metrics = ["test_f1_micro", "test_f1_macro"] + [f"test_f1_class_{i}" for i in range(5)]  # Adjust range based on num_classes

avg_results = {}
for metric in metrics:
    scores = [r[metric] for r in [x.metrics for x in all_results]]
    avg_results[metric] = {
        'mean': np.mean(scores),
        'std': np.std(scores)
    }

# Print results
for metric in metrics:
    print(f"Average {metric}: {avg_results[metric]['mean']:.4f} ± {avg_results[metric]['std']:.4f}")

Average test_f1_micro: 0.7713 ± 0.0087
Average test_f1_macro: 0.7384 ± 0.0157
Average test_f1_class_0: 0.8289 ± 0.0058
Average test_f1_class_1: 0.7044 ± 0.0346
Average test_f1_class_2: 0.7935 ± 0.0088
Average test_f1_class_3: 0.6558 ± 0.0142
Average test_f1_class_4: 0.7091 ± 0.0527
